<a href="https://colab.research.google.com/github/Yiding5805/Econnometrics_2021/blob/main/M4_Active_IV2SLS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Briefings**

1. Two packages:


*   https://www.statsmodels.org/dev/generated/statsmodels.sandbox.regression.gmm.IV2SLS.html
*   https://bashtage.github.io/linearmodels/doc/iv/examples/advanced-examples.html

2. Four variables:

Dependent Variables: Y
Endogenous Variable: X
Control variables: W
instruments: Z

3. Two regressions


*   First: X~Z
*   Second: Y~(X+W)





background readings: https://cryptobriefing.com/earning-20-apy-with-yield-app/

# Data Preparations

### Date Sources 
https://docs.defipulse.com/overview-of-api-endpoints

https://data-beta.defipulse.com/dashboard/egs

https://docs.defipulse.com/api-docs-by-provider/defi-pulse-data/total-value-locked/total-value-locked

### Import the Historical Data of Total Value Locked

In [ ]:
import pandas as pd
import requests
import json 
from datetime import datetime
TVL= requests.get('https://data-api.defipulse.com/api/v1/defipulse/api/GetHistory?resolution=history&project=all&api-key=8f22b45c31ddc56f5723f099ba3a2c1bd02f8e45a777eb4b13e4f2b2d99b')
TVL= TVL.json()
TVL=pd.DataFrame(TVL)
TVL.head()

,timestamp,tvlUSD,tvlETH,BTC,ETH,DAI
0,1624420800,47229560516,2.342527e+07,146244.300041,8.457771e+06,2.324961e+09
1,1624406400,47357581487,2.520817e+07,160834.838905,8.494428e+06,2.511823e+09
2,1624320000,48201277920,2.542972e+07,163439.102616,8.594571e+06,2.561610e+09
3,1624233600,53182280446,2.361809e+07,155975.628396,8.656196e+06,2.631900e+09
4,1624147200,54729515748,2.511012e+07,172738.968039,8.681611e+06,2.843341e+09


### Epoch & Unix Timestamp Conversion Tools: 

https://www.epochconverter.com/

In [ ]:
import datetime
TVL['timestamp']=TVL['timestamp'].astype(int)
TVL.dtypes

timestamp      int64
tvlUSD         int64
tvlETH       float64
BTC          float64
ETH          float64
DAI          float64
dtype: object

In [ ]:
TVL['time']=TVL['timestamp'].apply(lambda x: datetime.datetime.utcfromtimestamp(x).replace(tzinfo=datetime.timezone.utc))
TVL.head()

,timestamp,tvlUSD,tvlETH,BTC,ETH,DAI,time
0,1624420800,47229560516,2.342527e+07,146244.300041,8.457771e+06,2.324961e+09,2021-06-23 04:00:00+00:00
1,1624406400,47357581487,2.520817e+07,160834.838905,8.494428e+06,2.511823e+09,2021-06-23 00:00:00+00:00
2,1624320000,48201277920,2.542972e+07,163439.102616,8.594571e+06,2.561610e+09,2021-06-22 00:00:00+00:00
3,1624233600,53182280446,2.361809e+07,155975.628396,8.656196e+06,2.631900e+09,2021-06-21 00:00:00+00:00
4,1624147200,54729515748,2.511012e+07,172738.968039,8.681611e+06,2.843341e+09,2021-06-20 00:00:00+00:00


In [ ]:
TVL['date'] = pd.to_datetime(TVL['time']).dt.date
TVL.head()

,timestamp,tvlUSD,tvlETH,BTC,ETH,DAI,time,date
0,1624420800,47229560516,2.342527e+07,146244.300041,8.457771e+06,2.324961e+09,2021-06-23 04:00:00+00:00,2021-06-23
1,1624406400,47357581487,2.520817e+07,160834.838905,8.494428e+06,2.511823e+09,2021-06-23 00:00:00+00:00,2021-06-23
2,1624320000,48201277920,2.542972e+07,163439.102616,8.594571e+06,2.561610e+09,2021-06-22 00:00:00+00:00,2021-06-22
3,1624233600,53182280446,2.361809e+07,155975.628396,8.656196e+06,2.631900e+09,2021-06-21 00:00:00+00:00,2021-06-21
4,1624147200,54729515748,2.511012e+07,172738.968039,8.681611e+06,2.843341e+09,2021-06-20 00:00:00+00:00,2021-06-20


In [ ]:
TVL=TVL.drop(columns='time')
TVL['date']=pd.to_datetime(TVL['date'])
TVL.dtypes

timestamp             int64
tvlUSD                int64
tvlETH              float64
BTC                 float64
ETH                 float64
DAI                 float64
date         datetime64[ns]
dtype: object

### Import the data for Ethereum Prices

[Data Metrics](https://github.com/coinmetrics-io/data/blob/master/csv/metrics.csv)

In [ ]:
df_eth=pd.read_csv("https://raw.githubusercontent.com/coinmetrics-io/data/master/csv/eth.csv",usecols=(["time","AdrActCnt","CapMrktCurUSD","TxTfrValAdjUSD","AdrBalCnt","NVTAdj90","SplyCur"]))
df_eth['time']=pd.to_datetime(df_eth['time'])
df_eth.dtypes

time              datetime64[ns]
AdrActCnt                float64
AdrBalCnt                float64
CapMrktCurUSD            float64
NVTAdj90                 float64
SplyCur                  float64
TxTfrValAdjUSD           float64
dtype: object

### Merge the two Data Frame

In [ ]:
df =df_eth.merge(TVL, left_on='time', right_on='date')
df.head()

,time,AdrActCnt,AdrBalCnt,CapMrktCurUSD,NVTAdj90,SplyCur,TxTfrValAdjUSD,timestamp,tvlUSD,tvlETH,BTC,ETH,DAI,date
0,2017-08-02,126128.0,2563959.0,2.039684e+10,20.362229,9.374206e+07,6.780871e+08,1501632000,0,0.000000,0.0,0.0,0.0,2017-08-02
1,2017-08-03,132391.0,2583277.0,2.100563e+10,20.979627,9.376303e+07,6.062566e+08,1501718400,4,0.018339,0.0,0.0,0.0,2017-08-03
2,2017-08-04,135064.0,2605667.0,2.077052e+10,20.714810,9.378433e+07,5.206240e+08,1501804800,4,0.017826,0.0,0.0,0.0,2017-08-04
3,2017-08-05,134502.0,2624386.0,2.335192e+10,23.205816,9.380552e+07,6.665590e+08,1501891200,5,0.022665,0.0,0.0,0.0,2017-08-05
4,2017-08-06,135792.0,2640032.0,2.496085e+10,24.667311,9.382679e+07,9.229966e+08,1501977600,5,0.019756,0.0,0.0,0.0,2017-08-06


In [ ]:
df=df[["date","tvlUSD","AdrActCnt","SplyCur","AdrBalCnt"]]

# Test Instrument Validities

Y: "tvlUSD"

X: "AdrActCnt"

Z: "SplyCur"

W: "AdrBalCnt"

In [ ]:
df[["AdrActCnt","SplyCur"]].corr()

,AdrActCnt,SplyCur
AdrActCnt,1.000000,0.610777
SplyCur,0.610777,1.000000


# Using the Linear Model Package

https://bashtage.github.io/linearmodels/doc/iv/index.html

In [ ]:
pip install linearmodels

     |████████████████████████████████| 1.5MB 12.9MB/s 
     |████████████████████████████████| 9.5MB 32.2MB/s 
  Found existing installation: statsmodels 0.10.2
    Uninstalling statsmodels-0.10.2:
      Successfully uninstalled statsmodels-0.10.2


In [ ]:
from linearmodels import IV2SLS

### run an OLS

In [ ]:
df['const'] = 1

In [ ]:
ivolsmod = IV2SLS(df.tvlUSD, df[["const","AdrActCnt","SplyCur","AdrBalCnt"]], None, None)
res_ols = ivolsmod.fit()
print(res_ols)

                            OLS Estimation Summary                            
Dep. Variable:                 tvlUSD   R-squared:                      0.7349
Estimator:                        OLS   Adj. R-squared:                 0.7343
No. Observations:                1420   F-statistic:                    1471.0
Date:                Wed, Jun 23 2021   P-value (F-stat)                0.0000
Time:                        04:18:49   Distribution:                  chi2(3)
Cov. Estimator:                robust                                         
                                                                              
                             Parameter Estimates                              
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
const       6.443e+11  2.289e+10     28.149     0.0000   5.994e+11   6.891e+11
AdrActCnt    1.39e+04     3033.7     4.5809     0.00

### run an IV2SLS

In [ ]:
ivmod = IV2SLS(df.tvlUSD, df[["const","AdrBalCnt"]], df["AdrActCnt"], df["SplyCur"])
res_2sls = ivmod.fit()
print(res_2sls.summary)

                          IV-2SLS Estimation Summary                          
Dep. Variable:                 tvlUSD   R-squared:                      0.4235
Estimator:                    IV-2SLS   Adj. R-squared:                 0.4227
No. Observations:                1420   F-statistic:                    1495.9
Date:                Wed, Jun 23 2021   P-value (F-stat)                0.0000
Time:                        04:18:49   Distribution:                  chi2(2)
Cov. Estimator:                robust                                         
                                                                              
                             Parameter Estimates                              
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
const      -3.873e+10  1.113e+09    -34.813     0.0000  -4.091e+10  -3.655e+10
AdrBalCnt     -99.117     44.631    -2.2208     0.02

### comparison of the results

In [ ]:
from collections import OrderedDict
from linearmodels.iv.results import compare

In [ ]:
res = OrderedDict()
res['OLS'] = res_ols
res['2SLS'] = res_2sls
print(compare(res))

                 Model Comparison                
                               OLS           2SLS
-------------------------------------------------
Dep. Variable               tvlUSD         tvlUSD
Estimator                      OLS        IV-2SLS
No. Observations              1420           1420
Cov. Est.                   robust         robust
R-squared                   0.7349         0.4235
Adj. R-squared              0.7343         0.4227
F-statistic                 1471.0         1495.9
P-value (F-stat)            0.0000         0.0000
==================     ===========   ============
const                    6.443e+11     -3.873e+10
                          (28.149)      (-34.813)
AdrActCnt                 1.39e+04      1.282e+05
                          (4.5809)       (24.361)
SplyCur                    -6987.5               
                         (-28.821)               
AdrBalCnt                   3337.6        -99.117
                          (29.834)      (-2.2208)


### Testing endogeneity

The Durbin test is a classic of endogeneity which compares OLS estimates with 2SLS and exploits the fact that OLS estimates will be relatively efficient. Durbin’s test is not robust to heteroskedasticity.

In [ ]:
res_2sls.durbin()

Durbin test of exogeneity
H0: All endogenous variables are exogenous
Statistic: 451.6693
P-value: 0.0000
Distributed: chi2(1)
WaldTestStatistic, id: 0x7f18c9a3ce90

Wooldridge’s regression-based test of exogeneity is robust to heteroskedasticity since it inherits the covariance estimator from the model. Here there is little difference.

In [ ]:
res_2sls.wooldridge_regression

Wooldridge's regression test of exogeneity
H0: Endogenous variables are exogenous
Statistic: 830.6681
P-value: 0.0000
Distributed: chi2(1)
WaldTestStatistic, id: 0x7f18a9358dd0

### print first stage regression

In [ ]:
print(res_2sls.first_stage)

    First Stage Estimation Results    
                             AdrActCnt
--------------------------------------
R-squared                       0.6901
Partial R-squared               0.3971
Shea's R-squared                0.3971
Partial F-statistic             829.75
P-value (Partial F-stat)        0.0000
Partial F-stat Distn           chi2(1)
========================== ===========
const                        5.975e+06
                              (29.258)
AdrBalCnt                       0.0301
                              (39.433)
SplyCur                        -0.0611
                             (-28.805)
--------------------------------------

T-stats reported in parentheses
T-stats use same covariance type as original model


# Using the Statsmodel Package

Not recommended

https://www.statsmodels.org/dev/generated/statsmodels.sandbox.regression.gmm.IV2SLS.html